In [ ]:
import torch

In [ ]:
# create a 3 layer MLP network
class MLP(torch.nn.Module):
    def __init__(self, input_size, output_size, hidden_size):
        super(MLP, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.linear1 = torch.nn.Linear(input_size, hidden_size)
        self.linear2 = torch.nn.Linear(hidden_size, hidden_size)
        self.linear3 = torch.nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.linear1(x)
        x = torch.nn.functional.relu(x)
        x = self.linear2(x)
        x = torch.nn.functional.relu(x)
        x = self.linear3(x)
        return x

In [ ]:
# generate training data from function y = x1/(x2+0.5)
def generate_data(num_data):
    x = torch.rand(num_data, 2)
    y = x[:, 0]/(x[:, 1]+0.5)
    return x, y

In [ ]:
# generate training data
num_data = 10000
x, y = generate_data(num_data)
x = x.to('cuda')
y = y.to('cuda')

In [ ]:
# train network
model = MLP(2, 1, 128).to('cuda')
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
batch_size = 4
num_epochs = 1000
for epoch in range(num_epochs):
    for i in range(num_data//batch_size):
        x_batch = x[i*batch_size:(i+1)*batch_size, :]
        y_batch = y[i*batch_size:(i+1)*batch_size]
        y_pred = model(x_batch)
        loss = criterion(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if epoch % 1 == 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))